<a href="https://colab.research.google.com/github/C-Keat/Reading-Uni-Course-Work/blob/main/Cloud_Computing_CW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cloud Computing Course work workspace

In this Google Colab enviroment we create the code needed to perform the tasks requesed from the assignement.

* Task 1: Determine the total number of flights from each airport
* Task 2: Determine the passenger having had the highest number of flights.

For this task in the development process, develop a MapReduce-like executable prototype, (in Java, C, C++, or Python). The objective is to develop the basic functional ‘building-blocks’ that will address the Tasks 1 & 2 as
listed above, in a way that emulates the MapReduce/Hadoop framework.

The solution may use multi-threading as required. The marking scheme reflects the appropriate use of coding techniques, succinct code comments as required, data structures and overall program design. 

The code should be subject to version control best-practices using a hosted repository under your university username.

##The data, Class Creation & Muti-Threading Creation

First we will import the data and examine it.

In this assignment it was made clear that standard EDA and data cleaning did not need to take place, thus this is why it is not present in this workbook. In any normal situation checks and exploration would be done to make sure that the data is sound.

###Data inport

Here is where the data is imported

In [1]:
#importing libaries
import numpy as np
import pandas as pd
import csv

In [2]:
#Importing the data into dataframes
passengerDataOrgin_df = pd.read_csv("/content/drive/MyDrive/Education/Universitys/Reading University/Masters/Course/S2/Big Data and Cloud Computing (2021 22)/Cloud Computing/Assignment/Data/AComp_Passenger_data_no_error(1).csv", header=None)

Display the head of each of the dataframes for validation of import.

In [3]:
passengerDataOrgin_df

,0,1,2,3,4,5
0,UES9151GS5,SQU6245R,DEN,FRA,1420564460,1049
1,UES9151GS5,XXQ4064B,JFK,FRA,1420563917,802
2,EZC9678QI6,SOH3431A,ORD,MIA,1420563649,250
3,ONL0812DH1,SOH3431A,ORD,MIA,1420563649,250
4,CYJ0225CH1,PME8178S,DEN,PEK,1420564409,1322
...,...,...,...,...,...,...
495,BWI0520BG6,BER7172M,KUL,LAS,1420565167,1848
496,LLZ3798PE3,EWH6301Y,CAN,DFW,1420564967,1683
497,KKP5277HZ7,KJR6646J,IAH,BKK,1420565203,1928
498,JJM4724RF7,XXQ4064B,JFK,FRA,1420563917,802


###Read/Parse input data

Here we read the data in and transform it into the input data for the mapper phase.

In [4]:
#Complete parse of data
testFile = open("/content/drive/MyDrive/Education/Universitys/Reading University/Masters/Course/S2/Big Data and Cloud Computing (2021 22)/Cloud Computing/Assignment/Data/AComp_Passenger_data_no_error(1).csv")
#using csvreader object to read the csv file
csvreader = csv.reader(testFile)
#loop through the rows of the csv file and add them to the input data for the mapper to do its thing.

passData = []
for row in csvreader:
  passData.append(row)
passData
testFile.close()

In [ ]:
#prints the full listed data from the CSV file.
passData

###The Mapper & Reducer classes 

Here we implement the mapper Class and create an object to be used later.
We do the same for the reducer.

####The Mapper

Here is the mapper class.

In [6]:
#creation of the mapper class
class Mapper(object):
  
  def __init__(self):
    self.output = []
    
  def mapper(self,dataset, index):
      self.output = []
      for line in dataset:
        self.output.append(tuple((line[index],1)))
      return self.output
    

In [7]:
fmapper = Mapper()

####The Reducer

Here is the reducer class and its child class.

In [8]:
#creating the reducer class
class Reducer(object):
  
  def __init__(self):
     self.c="" 
     self.counter = {}
  
  def reducer(self,input):
  #searches line by line counting keyvalues.
    for line in input:
      if line[0] not in self.counter:
        self.counter[line[0]] = 0
      self.counter[line[0]] += 1
    return self.counter


In [9]:
#this should have created an object of the reducer
fReducer = Reducer()

In [10]:
#create a child of the orginal reducer class, this is done purly to demonstrate understanding of the process of inheratiance in code.  
class HighestValueReducer(Reducer):

  def reducerHighestVal(self,input):
    self.counter = Reducer.reducer(self,input)
    
    max_key = max(self.counter, key = self.counter.get)
    #returns the most traveled customer
    return max_key

In [11]:
#Creating a object of the reducer child for use in task 2
highReducer = HighestValueReducer()

###MultiThreading implementation for mapper stage.

In this section we implement multithreading so that the mapper class can run more efficiently across chunks of data. We do this by first taking the original amount of data and break it into a specified amount of chunks then, starting threads of the mapper for each of the chunks. The output of the mapper is then joined before being passed to the reducer.

In [12]:
from threading import Thread

####Multi-Threading class and function defined

Here we define the threading class that allows us to return an output from a thread.

In [13]:
#class that allows for threads to return things
class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None
    def run(self):
        print(type(self._target))
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

In [14]:
#Here we denfine the mapping multi-Thread function.
def threadMapper(data, index, chunks):
  
  #Split the orgin data into list of chunks
  n = int(len(data)/chunks)
  output=[data[i:i + n] for i in range(0, len(data), n)]
 
  fullList = []
  for i in range(len(output)):
    #creates threads based on the number of chunks to perform the mapper task.
    tmrvt = ThreadWithReturnValue(target=fmapper.mapper,args=(output[i],index))
    tmrvt.start()
    fullList.append(tmrvt.join())
  
  finalList = []
  for line in fullList:
    finalList = finalList + line

  
  return finalList

##Task 1

Determine the total number of flights from each airport

###Performing the mapper function for Task 1

In [15]:
#starts the threading to produce the full keyvalue list for task 1
fullListPorts = threadMapper(passData,2,4)

<class 'method'>
<class 'method'>
<class 'method'>
<class 'method'>


In [ ]:
#Prints the full list after the mapper step has been complete
fullListPorts

###The reducer

Here we implement the reducer and allow it to find our outputs.

In [17]:
#performs a function inside the class
finalOutput = fReducer.reducer(fullListPorts)

###The Output

Here we show the output of the map-reduce funcinality.

This should represent the area that displays the outputs that the assignment is looking for.

In [18]:
#finalOutput represents the total number of flights from each airport
finalOutput

{'AMS': 15,
 'ATL': 36,
 'BKK': 17,
 'CAN': 37,
 'CDG': 21,
 'CGK': 27,
 'CLT': 21,
 'DEN': 46,
 'DFW': 11,
 'FCO': 15,
 'HND': 13,
 'IAH': 37,
 'JFK': 25,
 'KUL': 33,
 'LAS': 17,
 'LHR': 25,
 'MAD': 13,
 'MIA': 11,
 'MUC': 14,
 'ORD': 33,
 'PEK': 13,
 'PVG': 20}

##Task 2

Here is task 2. 

Determine the passenger having had the highest number of flights.

####Performing the mapper for Task 2

In [19]:
#Produce a full list of mapped keyvalues for task 2
fullListPeople = threadMapper(passData,0,4)

<class 'method'>
<class 'method'>
<class 'method'>
<class 'method'>


In [ ]:
fullListPeople 

###Reduce

Create the reduce function.

In [21]:
#perform reducer step on keyvalue pairs for task 2, using the child object of reducer class
mostTraveledCustomer = highReducer.reducerHighestVal(fullListPeople)

###Output

Here represents the output of this task.

In [22]:
#the mostTraveledCustomer represents the customer that travels the most 
mostTraveledCustomer

'UES9151GS5'